# Суммарные показатели по брендам
Пример расчета суммарных показателей брендов 

## Описание задачи и условий расчета
- Период: 01.09.2022-30.09.2022
- ЦА: Все 18-59
- Место просмотра: Все места (Дом+Дача)
- Каналы: BRIDGE HITS, BRIDGE ШЛЯГЕР, MUSIC BOX RUSSIA, АВТО ПЛЮС ТВ, АНЕКДОТ ТВ, ПРОДВИЖЕНИЕ, РЕТРО, РУССКИЙ ИЛЛЮЗИОН, УСАДЬБА, ФЕНИКС+ КИНО
- Ролики: тип - Ролик
- Статистики: Quantity (тотал), CumReach%, Rtg% (тотал), StandRtg% (30) (тотал)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем нужные нам переменные
cats.get_tv_demo_attribute(names=['возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: age

In [ ]:
# Получим id для типа рекламы - ролик
cats.get_tv_ad_type(name=['ролик'])

In [ ]:
# Отберем нужные нам каналы в справочнике телекомпаний и выведем их id
cats.get_tv_company(name=[
    "BRIDGE HITS (СЕТЕВОЕ ВЕЩАНИЕ)",
    "BRIDGE ШЛЯГЕР (СЕТЕВОЕ ВЕЩАНИЕ)",
    "MUSIC BOX RUSSIA (СЕТЕВОЕ ВЕЩАНИЕ)",
    "АВТО ПЛЮС ТВ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "АНЕКДОТ ТВ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "ПРОДВИЖЕНИЕ (СЕТЕВОЕ ВЕЩАНИЕ)",
    "РЕТРО (СЕТЕВОЕ ВЕЩАНИЕ)",
    "РУССКИЙ ИЛЛЮЗИОН (СЕТЕВОЕ ВЕЩАНИЕ)",
    "УСАДЬБА (СЕТЕВОЕ ВЕЩАНИЕ)",
    "ФЕНИКС+ КИНО (СЕТЕВОЕ ВЕЩАНИЕ)"])['id'].tolist()

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА: Все 18-59
basedemo_filter = 'age >= 18 and age <= 59'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы: укажем id телекомпаний, которые мы ранее нашли в справочнике
company_filter = 'tvCompanyId IN (2335, 2795, 2796, 2805, 3204, 3205, 7088, 12398, 12809, 13137)'

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов: тип - ролик
ad_filter = 'adTypeId = 1'

# Указываем список срезов (группировок)
slices = [
    'tvCompanyName', #Телекомпания
    'advertiserName', #Рекламодатель
    'brandName', #Бренд
    'brandListName', #Список брендов
]

# Указываем список статистик для расчета
statistics = ['QuantitySum','CumReachPer','RtgPerSum','StandRtgPerSum']

# Задаем условия сортировки: Телекомпания (от а до я), рекламодатель (от а до я), Бренд (от а до я)
sortings = {'tvCompanyName':'ASC','advertiserName':'ASC','brandName':'ASC','brandListName':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 4, #TV Index Plus All Russia
    "issueType": "AD", #Тип события - Ролики
    "standRtg" : {
      "useRealDuration" : True, #расчет по реальной длительности ролика.
      "standardDuration" : 20 #стандартная длительность 20 сек.
    }
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_crosstab = mtask.wait_task(mtask.send_crosstab_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_crosstab))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('06_sum_stats_by_brand')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)